In [1]:
import numpy as np
import cvxpy as cp
np.set_printoptions(precision=2, suppress=True)

## Linear program formulation
$ \max{\sum_t{1(wood_t - swood_{t+1}) + 2(stone_t - sstone_{t+1}) + 3(iron_t - siron_{t+1}) + 5(tools_t + tools_{t+1}) + 10score\_housing_t}} $


**Constraints**

<u> The resource amount we are left with at each turn must be valid </u>
$\\ stockpiles, production \ge 0$
$ \\ wood_t = pwood_t + swood_t - wood\_cost_t \ge 0$
$ \\ stone_t = pstone_t + sstone_t - stone\_cost_t \ge 0$
$ \\ iron_t = piron_t + siron_t - iron\_cost_t \ge 0$
$ \\ tools_t = ptools_t + stools_t - tools\_cost_t \ge 0$

<u> Stockpile capacity </u>
$ \\ swood_t + sstone_t + siron_t + stools_t \le 30 $

<u> Initial stockpile </u>
$ \\ swood_1, sstone_1, siron_1, stools_1 = 5 $

<u> The future stockpile can stock only up to the amount of the currently available resource </u>
$ \\ swood_{t+1} \le wood_t $
$ \\ sstone_{t+1} \le stone_t $
$ \\ siron_{t+1} \le iron_t $
$ \\ stools_{t+1} \le tools_t $

<u> Mining constraint </u>
$ \\ stone_1 \le 5 $
$ \\ stone_{t+1} \le 5 + 5 \cdot mining_t $

<u> Labour constraint </u>
$ \\ labour\_cost_1 \le 50 $
$ \\ labour\_cost_{t+1} \le 50 + 10 \cdot labour\_housing_t $

**Variables**

$
stockpiles = \begin{bmatrix} swood \space sstone \space siron \space stools \end{bmatrix}^T
$

$
production = \begin{bmatrix} pwood \space pstone \space piron \space ptools \space mining \space labour\_housing \space score\_housing \end{bmatrix}^T
$

In [2]:
stockpile = cp.Variable((4, 5), nonneg=True)
production = cp.Variable((6, 5), nonneg=True)
points = cp.Variable((5, 5), nonneg=True)

In [3]:
costs = np.array([
	[0, 0, 0, 0.1, 1], # wood
	[0, 0, 0, 0.1, 3], # stone
	[0, 0, 0, 0.1, 2], # iron
	[1, 0, 1, 0.1, 1], # tools
	[2, 3, 0, 0.2, 2], # housing
	[0, 0, 0, 0.2, 1], # mining
])

scores = np.array([1, 2, 3, 5, 10])

In [4]:
resource_cost = (costs.T @ production)[:4]
labour_cost = (costs.T @ production)[4]

In [5]:
resource_production = production[:4]
housing, mining = production[4:]
iron_production = production[2]
resource_points = points[:4]
housing_points = points[4]

In [6]:
constraints = [
    # Sell only resource in possession
    points <= production[:5],

    # Use only the stockpiled resource
    resource_cost <= stockpile,

    # Stockpile capacity
    stockpile.sum(axis=0) <= 30,
    
    # Initialize stockpile
    stockpile.T[0] == 5,
    
    # The future stockpile can stock only up to the amount of the currently available resource
    stockpile.T[1:] == resource_production.T[:4] - points[:4].T[:4],

    # Labour constraints
    labour_cost[0] <= 50,
	labour_cost[1:] <= 50 + 10*(housing - housing_points)[:4].cumsum(),

    # Mining constraints
    iron_production[0] <= 5,
    iron_production[1:] <= 5 + mining[:4].cumsum()
]


In [7]:
cp.Problem(cp.Maximize((scores @ points).sum()), constraints).solve()

/home/ventus/.local/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


310.47794115863434